In [9]:
import pandas as pd
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import GaussianNB
from imblearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_score
from yellowbrick.classifier.rocauc import roc_auc
from yellowbrick.classifier import ROCAUC
from imblearn.over_sampling import SMOTENC
from sklearn.metrics import roc_curve, auc
import random
random.seed(2814)

In [10]:
X_train = pd.read_csv('../data/interim/1_X_train.csv')
X_test = pd.read_csv('../data/interim/1_X_test.csv')
y_train = pd.read_csv('../data/interim/1_y_train.csv')
y_test = pd.read_csv('../data/interim/1_y_test.csv')

In [11]:
X_train

,GroundFloorArea(sq m),InsulationThickness,TotalDeliveredEnergy,YearofConstruction_1753,YearofConstruction_1757,YearofConstruction_1759,YearofConstruction_1760,YearofConstruction_1761,YearofConstruction_1762,YearofConstruction_1764,...,StructureType_Masonry,StructureType_Timber or Steel Frame,InsulationType_Factory Insulated,InsulationType_Loose Jacket,InsulationType_None,NoOfSidesSheltered_four,NoOfSidesSheltered_one,NoOfSidesSheltered_three,NoOfSidesSheltered_two,NoOfSidesSheltered_zero
0,0.041128,0.016026,0.001022,0,0,0,0,0,0,0,...,0,1,1,0,0,0,0,0,1,0
1,0.038657,0.000000,0.011529,0,0,0,0,0,0,0,...,1,0,0,1,0,0,0,0,1,0
2,0.032150,0.018697,0.003419,0,0,0,0,0,0,0,...,0,1,1,0,0,0,0,0,1,0
3,0.046305,0.023504,0.004289,0,0,0,0,0,0,0,...,1,0,1,0,0,0,0,1,0,0
4,0.050612,0.000000,0.010762,0,0,0,0,0,0,0,...,1,0,0,0,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1494355,0.007039,0.000000,0.001751,0,0,0,0,0,0,0,...,1,0,0,0,1,0,0,1,0,0
1494356,0.019839,0.012461,0.007560,0,0,0,0,0,0,0,...,1,0,1,0,0,0,0,0,1,0
1494357,0.010572,0.000000,0.004070,0,0,0,0,0,0,0,...,1,0,0,0,1,0,0,1,0,0
1494358,0.020324,0.013393,0.013289,0,0,0,0,0,0,0,...,1,0,1,0,0,0,0,0,0,1


In [12]:
# Get a stratified sample of training data to train the model with. Do not touch testing data
df = pd.concat([X_train, y_train], axis = 1)

In [29]:
df_sample = df.groupby('EnergyRating', group_keys=False).apply(lambda x: x.sample(1334))

In [30]:
X_train = df_sample.iloc[:, :-1] # Independent Variables
y_train = df_sample.iloc[:, -1] # Dependent Variables

In [31]:
X_train

,GroundFloorArea(sq m),InsulationThickness,TotalDeliveredEnergy,YearofConstruction_1753,YearofConstruction_1757,YearofConstruction_1759,YearofConstruction_1760,YearofConstruction_1761,YearofConstruction_1762,YearofConstruction_1764,...,StructureType_Masonry,StructureType_Timber or Steel Frame,InsulationType_Factory Insulated,InsulationType_Loose Jacket,InsulationType_None,NoOfSidesSheltered_four,NoOfSidesSheltered_one,NoOfSidesSheltered_three,NoOfSidesSheltered_two,NoOfSidesSheltered_zero
863822,0.027430,0.009425,0.001277,0,0,0,0,0,0,0,...,1,0,1,0,0,1,0,0,0,0
881547,0.024623,0.015358,0.001165,0,0,0,0,0,0,0,...,1,0,1,0,0,0,0,0,1,0
839679,0.018316,0.018697,0.001096,0,0,0,0,0,0,0,...,1,0,1,0,0,0,0,1,0,0
808707,0.083107,0.015549,0.001721,0,0,0,0,0,0,0,...,0,0,1,0,0,0,1,0,0,0
833818,0.098309,0.013338,0.001815,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1494283,0.011621,0.013601,0.004803,0,0,0,0,0,0,0,...,1,0,1,0,0,0,0,0,1,0
39398,0.012577,0.000000,0.002868,0,0,0,0,0,0,0,...,1,0,0,0,1,1,0,0,0,0
1472929,0.012887,0.026709,0.004642,0,0,0,0,0,0,0,...,1,0,0,1,0,1,0,0,0,0
458722,0.018435,0.042735,0.003943,0,0,0,0,0,0,0,...,1,0,1,0,0,1,0,0,0,0


In [50]:
# Modelling
dt = DecisionTreeClassifier(criterion='entropy', max_depth=20, min_samples_leaf=3, min_samples_split=9, random_state=2814)
dt.fit(X_train, y_train.values.ravel())

y_pred = dt.predict(X_test)
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

          A1       0.08      0.73      0.14       252
          A2       0.74      0.65      0.69      8648
          A3       0.76      0.63      0.69     10239
          B1       0.34      0.60      0.43      3052
          B2       0.49      0.56      0.52      6563
          B3       0.69      0.64      0.67     15594
          C1       0.73      0.73      0.73     22757
          C2       0.74      0.72      0.73     24906
          C3       0.73      0.71      0.72     23637
          D1       0.74      0.71      0.73     22871
          D2       0.76      0.75      0.76     19637
          E1       0.67      0.72      0.69     11326
          E2       0.69      0.71      0.70      8956
           F       0.75      0.76      0.76      9269
           G       0.93      0.88      0.90     13363

    accuracy                           0.71    201070
   macro avg       0.65      0.70      0.66    201070
weighted avg       0.73   

In [43]:
param_dict = {
    'criterion': ['gini', 'entropy']
    ,'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
    ,'min_samples_leaf': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
    ,'min_samples_split': [2, 3, 4, 5, 6, 7, 8, 9, 10]
}

In [44]:
grid = GridSearchCV(
    dt
    ,param_dict
    ,cv=5
    ,verbose=1
    ,n_jobs=-1
)

In [45]:
grid.fit(X_train, y_train)

Fitting 5 folds for each of 1800 candidates, totalling 9000 fits


GridSearchCV(cv=5,
             estimator=DecisionTreeClassifier(criterion='entropy', max_depth=30,
                                              min_samples_leaf=5,
                                              random_state=2814),
             n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100],
                         'min_samples_leaf': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
                         'min_samples_split': [2, 3, 4, 5, 6, 7, 8, 9, 10]},
             verbose=1)

In [48]:
grid.best_params_

{'criterion': 'entropy',
 'max_depth': 20,
 'min_samples_leaf': 3,
 'min_samples_split': 9}

In [49]:
grid.best_score_

0.7128435782108945

In [35]:
rf = RandomForestClassifier(random_state=2814)
rf.fit(X_train, y_train.values.ravel())
y_pred = rf.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

          A1       0.12      0.74      0.21       252
          A2       0.76      0.74      0.75      8648
          A3       0.76      0.68      0.72     10239
          B1       0.30      0.59      0.39      3052
          B2       0.28      0.42      0.34      6563
          B3       0.37      0.38      0.38     15594
          C1       0.39      0.35      0.37     22757
          C2       0.36      0.30      0.33     24906
          C3       0.34      0.33      0.34     23637
          D1       0.36      0.30      0.33     22871
          D2       0.37      0.32      0.34     19637
          E1       0.26      0.30      0.28     11326
          E2       0.24      0.30      0.27      8956
           F       0.33      0.40      0.36      9269
           G       0.68      0.71      0.70     13363

    accuracy                           0.40    201070
   macro avg       0.39      0.46      0.41    201070
weighted avg       0.41   

In [36]:
knn = KNeighborsClassifier()
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

          A1       0.04      0.72      0.08       252
          A2       0.62      0.63      0.62      8648
          A3       0.57      0.53      0.55     10239
          B1       0.09      0.42      0.15      3052
          B2       0.11      0.31      0.16      6563
          B3       0.17      0.22      0.20     15594
          C1       0.22      0.19      0.20     22757
          C2       0.21      0.16      0.18     24906
          C3       0.20      0.15      0.17     23637
          D1       0.21      0.13      0.16     22871
          D2       0.20      0.13      0.16     19637
          E1       0.13      0.14      0.14     11326
          E2       0.13      0.16      0.14      8956
           F       0.17      0.17      0.17      9269
           G       0.57      0.39      0.46     13363

    accuracy                           0.22    201070
   macro avg       0.24      0.30      0.24    201070
weighted avg       0.25   

In [37]:
lr = LogisticRegression(random_state=2814, solver='sag')
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

          A1       0.04      0.72      0.08       252
          A2       0.72      0.50      0.59      8648
          A3       0.70      0.63      0.66     10239
          B1       0.12      0.40      0.18      3052
          B2       0.16      0.30      0.21      6563
          B3       0.21      0.21      0.21     15594
          C1       0.26      0.23      0.25     22757
          C2       0.24      0.15      0.18     24906
          C3       0.23      0.19      0.21     23637
          D1       0.22      0.15      0.18     22871
          D2       0.20      0.17      0.19     19637
          E1       0.14      0.16      0.15     11326
          E2       0.12      0.18      0.14      8956
           F       0.15      0.24      0.19      9269
           G       0.51      0.60      0.55     13363

    accuracy                           0.25    201070
   macro avg       0.27      0.32      0.26    201070
weighted avg       0.27   

In [38]:
svm = LinearSVC(random_state=2814)
svm.fit(X_train, y_train)
y_pred = svm.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

          A1       0.04      0.73      0.07       252
          A2       0.70      0.48      0.57      8648
          A3       0.71      0.63      0.67     10239
          B1       0.14      0.37      0.20      3052
          B2       0.14      0.36      0.21      6563
          B3       0.21      0.19      0.20     15594
          C1       0.26      0.23      0.24     22757
          C2       0.23      0.13      0.17     24906
          C3       0.23      0.20      0.21     23637
          D1       0.22      0.14      0.17     22871
          D2       0.20      0.14      0.17     19637
          E1       0.13      0.12      0.13     11326
          E2       0.11      0.13      0.12      8956
           F       0.14      0.22      0.17      9269
           G       0.43      0.73      0.54     13363

    accuracy                           0.25    201070
   macro avg       0.26      0.32      0.26    201070
weighted avg       0.27   

In [39]:
nb = GaussianNB()
nb.fit(X_train, y_train)
y_pred = nb.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

          A1       0.00      0.08      0.00       252
          A2       0.41      0.97      0.58      8648
          A3       0.02      0.09      0.03     10239
          B1       0.02      0.06      0.03      3052
          B2       0.03      0.12      0.05      6563
          B3       0.05      0.04      0.05     15594
          C1       0.10      0.06      0.08     22757
          C2       0.10      0.06      0.07     24906
          C3       0.12      0.06      0.08     23637
          D1       0.08      0.03      0.04     22871
          D2       0.12      0.00      0.01     19637
          E1       0.17      0.01      0.02     11326
          E2       0.10      0.06      0.08      8956
           F       0.14      0.01      0.02      9269
           G       0.37      0.04      0.07     13363

    accuracy                           0.09    201070
   macro avg       0.12      0.11      0.08    201070
weighted avg       0.13   